<h1>Analytical Validation Tests for 2D FEM Solver</h1>

This notebook contains validation tests comparing numerical solutions against analytical results.

## Poiseuille Flow

Pressure-driven flow in $x$-direction with no-slip walls as $y$-boundary condition.

**Setup:**
- Body force $f_x$ drives flow in $x$-direction (equivalent to pressure gradient)
- Periodic boundary conditions in $x$ (that is why we cannot directly use pressure gradient to impose flow)
- No-slip walls at $y=0$ and $y=L_y$: $j_x = j_y = 0$
- In-plane viscous diffusion terms active (`R23xy`, `R23yx`)
- Wall stress terms $\tau_{xz}, \tau_{yz}$ deactivated (`R24x`, `R24y`)

**Analytical solution:**

$$u(y) = \frac{\rho \cdot f_x}{2\mu} \cdot (y - y_{lo}) \cdot (y_{hi} - y)$$

where $y_{\mathrm{lo}} = -\Delta y/2$ and $y_{\mathrm{hi}} = L_y + \Delta y/2$ account for BCs applied at ghost cell centers.

$\rightarrow$ **[YAML File](./examples/poiseuille_2d_body_force.yaml)**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from GaPFlow.problem import Problem

problem = Problem.from_yaml("examples/poiseuille_2d_body_force.yaml")
problem.q[0][:] = 1.0  # Uniform density
problem.q[1][:] = 0.0  # Zero momentum
problem.q[2][:] = 0.0
problem.run()

Compare with analytical solution:

In [ ]:
# Extract velocity profile
rho = problem.q[0][1:-1, 1:-1]
jx = problem.q[1][1:-1, 1:-1]
vx = (jx / rho).mean(axis=0)  # Average over x

# Parameters
Ly, Ny = problem.grid['Ly'], problem.grid['Ny']
dy = Ly / Ny
mu = problem.prop['shear']
f_x = problem.prop['force_x']
rho_m = rho.mean()

# Analytical solution (BCs at ghost cell centers)
y = np.linspace(dy/2, Ly - dy/2, Ny)
y_lo, y_hi = -dy/2, Ly + dy/2
u_analytical = (rho_m * f_x) / (2 * mu) * (y - y_lo) * (y_hi - y)

# Error
l2_error = np.sqrt(((vx - u_analytical)**2).mean()) / u_analytical.max()
print(f"L2 relative error: {l2_error:.2e}")

In [ ]:
# Plot comparison
fig, axes = plt.subplots(1, 3, figsize=(12, 4), facecolor='white', constrained_layout=True)

# 1. Velocity profile
axes[0].plot(u_analytical, y, 'b-', lw=2, label='Analytical')
axes[0].plot(vx, y, 'ro', ms=4, label='Simulated')
axes[0].set_xlabel('Velocity u [m/s]')
axes[0].set_ylabel('y [m]')
axes[0].set_title('Velocity Profile')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# 2. Error profile
rel_error = np.abs(vx - u_analytical) / u_analytical.max() * 100
axes[1].plot(rel_error, y, 'r-', lw=2)
axes[1].set_xlabel('Relative Error [%]')
axes[1].set_ylabel('y [m]')
axes[1].set_title('Error Profile')
axes[1].ticklabel_format(axis='x', style='scientific', scilimits=(-2, 2))
axes[1].grid(True, alpha=0.3)

# 3. 2D velocity field
Lx, Nx = problem.grid['Lx'], problem.grid['Nx']
vx_2d = (jx / rho)
x = np.linspace(dy/2, Lx - dy/2, Nx)
X, Y = np.meshgrid(x, y, indexing='ij')
im = axes[2].pcolormesh(X, Y, vx_2d, cmap='viridis', shading='auto')
axes[2].set_xlabel('x [m]')
axes[2].set_ylabel('y [m]')
axes[2].set_title('$v_x$ field')
plt.colorbar(im, ax=axes[2], label='[m/s]')

plt.suptitle('2D Poiseuille Flow Validation', fontweight='bold')
plt.show()

## Couette Flow

Shear-driven flow with stationary bottom wall and moving top wall.

**Setup:**
- Top wall moving with velocity $U$ (implemented as Dirichlet BC: $j_x = \rho \cdot U$)
- Bottom wall stationary: $j_x = j_y = 0$
- Periodic boundary conditions in $x$
- In-plane viscous diffusion terms active (`R23xy`, `R23yx`)
- No body force

**Analytical solution:**

$$u(y) = U \cdot \frac{y - y_{lo}}{y_{hi} - y_{lo}}$$

where $y_{\mathrm{lo}} = -\Delta y/2$ and $y_{\mathrm{hi}} = L_y + \Delta y/2$ account for BCs applied at ghost cell centers.

$\rightarrow$ **[YAML File](./examples/couette_2d_wall_velocity.yaml)**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from GaPFlow.problem import Problem

problem_couette = Problem.from_yaml("examples/couette_2d_wall_velocity.yaml")
problem_couette.q[0][:] = 1.0  # Uniform density
problem_couette.q[1][:] = 0.0  # Zero momentum
problem_couette.q[2][:] = 0.0
problem_couette.run()

Compare with analytical solution:

In [ ]:
# Extract velocity profile
rho_c = problem_couette.q[0][1:-1, 1:-1]
jx_c = problem_couette.q[1][1:-1, 1:-1]
vx_c = (jx_c / rho_c).mean(axis=0)  # Average over x

# Parameters
Ly_c, Ny_c = problem_couette.grid['Ly'], problem_couette.grid['Ny']
dy_c = Ly_c / Ny_c
rho_m_c = rho_c.mean()

# Wall velocity from BC (jx_wall / rho = U)
U_wall = problem_couette.grid['bc_yN_D_val'][1] / rho_m_c

# Analytical solution (linear profile, BCs at ghost cell centers)
y_c = np.linspace(dy_c/2, Ly_c - dy_c/2, Ny_c)
y_lo_c, y_hi_c = -dy_c/2, Ly_c + dy_c/2
u_analytical_c = U_wall * (y_c - y_lo_c) / (y_hi_c - y_lo_c)

# Error
l2_error_c = np.sqrt(((vx_c - u_analytical_c)**2).mean()) / u_analytical_c.max()
print(f"L2 relative error: {l2_error_c:.2e}")

In [ ]:
# Plot comparison
fig, axes = plt.subplots(1, 3, figsize=(12, 4), facecolor='white', constrained_layout=True)

# 1. Velocity profile
axes[0].plot(u_analytical_c, y_c, 'b-', lw=2, label='Analytical')
axes[0].plot(vx_c, y_c, 'ro', ms=4, label='Simulated')
axes[0].set_xlabel('Velocity u [m/s]')
axes[0].set_ylabel('y [m]')
axes[0].set_title('Velocity Profile')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# 2. Error profile
rel_error_c = np.abs(vx_c - u_analytical_c) / u_analytical_c.max() * 100
axes[1].plot(rel_error_c, y_c, 'r-', lw=2)
axes[1].set_xlabel('Relative Error [%]')
axes[1].set_ylabel('y [m]')
axes[1].set_title('Error Profile')
axes[1].ticklabel_format(axis='x', style='scientific', scilimits=(-2, 2))
axes[1].grid(True, alpha=0.3)

# 3. 2D velocity field
Lx_c, Nx_c = problem_couette.grid['Lx'], problem_couette.grid['Nx']
vx_2d_c = (jx_c / rho_c)
x_c = np.linspace(dy_c/2, Lx_c - dy_c/2, Nx_c)
X_c, Y_c = np.meshgrid(x_c, y_c, indexing='ij')
im = axes[2].pcolormesh(X_c, Y_c, vx_2d_c, cmap='viridis', shading='auto')
axes[2].set_xlabel('x [m]')
axes[2].set_ylabel('y [m]')
axes[2].set_title('$v_x$ field')
plt.colorbar(im, ax=axes[2], label='[m/s]')

plt.suptitle('2D Couette Flow Validation', fontweight='bold')
plt.show()